In [66]:
import getpass
import os
import pandas as pd
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.9,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## LLMChain

In [33]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [34]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Give only one very creative name."
)

In [35]:
chain = LLMChain(llm=llm, prompt=prompt)

In [36]:
product = "Queen Size Sheet Set"
chain.run(product)

'SlumberKingdom'

## SimpleSequentialChain

In [37]:
from langchain.chains import SimpleSequentialChain

In [54]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? You can Include Elements derrived from Hindi or Sanskrit to include in the name.\
    Generate a few names and shortlist 1 from them."
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [55]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [56]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [57]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are a few names for a company that makes queen-size sheet sets, incorporating Hindi/Sanskrit elements, followed by a shortlist:


**Name Ideas:**

* **Shayan Ratna:** (शयन रत्न)  "Shayan" (शयन) means sleep or bed, and "Ratna" (रत्न) means jewel or gem, implying luxurious sheets.
* **Nidra Suhag:** (निद्रा सुहाग) "Nidra" (निद्रा) means sleep, and "Suhag" (सुहाग) implies comfort and marital bliss (often associated with comfort and well-being).  This could be a slightly more evocative name.
* **Supta Shanti:** (सुप्ता शान्ति) "Supta" (सुप्ता) relates to sleep or lying down, and "Shanti" (शान्ति) means peace or tranquility.  This suggests restful sleep.
* **Rishi Rest:**  A simpler name combining a Sanskrit-derived word (Rishi, implying wisdom and serenity) with the English word "Rest."
* **Aaram Linen:** (आरम लिनन) "Aaram" (आरम) means comfort or rest, paired with the common term "Linen."


**Shortlist:**

I recommend **Shayan Ratna (शयन

'Shayan Ratna: Luxurious queen-size sheet sets, crafted for blissful, jewel-like sleep.'

## SequentialChain

In [58]:
from langchain.chains import SequentialChain

In [ ]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [60]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [61]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [62]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [63]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [69]:
from io import StringIO

In [70]:
str_df = 'Product,Review\nQueen Size Sheet Set,"I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets."\nWaterproof Phone Pouch,"I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch."\nLuxury Air Mattress,"This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it\'s the top of the mattress where it\'s kind of like fabric and a patch won\'t stick. Maybe I got unlucky with a defective mattress, but where\'s quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that\'s what the mattress was. Seriously horrible experience, ruined my friend\'s stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst."\nPillows Insert,"This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great"\n"Milk Frother Handheld\n","\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can\'t be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It\'s not overuse or abuse. I\'m very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck."\n"L\'Or Espresso Café\xa0\n","Je trouve le goût médiocre. La mousse ne tient pas, c\'est bizarre. J\'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"\nHervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento."\n'

In [71]:
df = pd.read_csv(StringIO(str_df))

In [74]:
df.head(6)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...


In [73]:
review = df.Review[5]
overall_chain(review)

/tmp/ipykernel_92383/1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The head doesn't hold, it's strange. I buy the same ones in stores and the taste is much better...\n\nOld batch or counterfeit!?",
 'summary': "The reviewer found the beer's taste inferior to store-bought versions, suspecting it might be old or counterfeit.",
 'followup_message': "Several options, depending on the desired tone:\n\n**Option 1 (Formal and polite):**\n\n> Nous vous remercions de votre commentaire.  Nous sommes désolés d'apprendre que la dégustation de notre bière ne vous a pas pleinement satisfait.  Nous prenons votre remarque concernant la possible altération ou contrefaçon très au sérieux et allons mener une enquête interne afin d'identifier la cause de ce problème.\n\n\n**Option 2 (Slightly less formal, more apologetic):**\n\n> Nous vous présentons

## Router Chain

In [76]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [77]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [78]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [79]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [90]:
destinations

['physics: Good for answering questions about physics',
 'math: Good for answering math questions',
 'History: Good for answering history questions',
 'computer science: Good for answering computer science questions']

In [80]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [81]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_92383/3710404048.py:1: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


In [82]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [83]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipykernel_92383/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [84]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation is the electromagnetic radiation emitted by an idealized object, a "black body," that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence.  This absorption leads to thermal equilibrium, and the object then emits radiation characteristic of its temperature. This emitted radiation follows a specific spectrum described by Planck\'s law, which shows the intensity of radiation at each wavelength is dependent solely on the object\'s temperature.'

In [85]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The question is: What is 2 + 2?\n\nThis is a simple addition problem.  The component parts are the two numbers, 2 and 2.  The operation is addition (+).\n\nAdding 2 and 2 results in 4.\n\nTherefore, 2 + 2 = 4'

In [86]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body (except for red blood cells, which are anucleate and thus lack a nucleus containing DNA, and some other highly specialized cells with limited function) contains DNA because DNA holds the **genetic instructions** for building and maintaining the organism.  These instructions are essential for:\n\n* **Protein synthesis:** DNA codes for the production of proteins, which are the workhorses of the cell.  Proteins perform virtually every function within a cell, from catalyzing metabolic reactions to providing structural support.\n\n* **Cell division and replication:**  When a cell divides (mitosis), its DNA must be accurately copied and passed on to the daughter cells.  This ensures that each new cell receives a complete set of instructions.\n\n* **Cellular differentiation:**  During development, cells specialize into different types (e.g., muscle cells, nerve cells).  The DNA determines which genes are expressed (turned on) and which are repressed (turned off), leadi